In [ ]:
sc

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
CLUSTER_ADD = "hdfs://ad-cluster-master:54310"
SOURCE_DIR = "/user/cluster/source_wfile/"
MODEL_DIR = "/user/cluster/"
PARQUET_SOURCE_FILE = "DFSaverBi.parquet"
PARQUET_SOURCE_DIR = "/user/cluster/"

num_cores = 10
gran = 3
Force_regen = False

#########################################################################
client = Config().get_client()


Parquet_List = client.list(PARQUET_SOURCE_DIR)


regen = False if (PARQUET_SOURCE_FILE in Parquet_List) else True


if regen == True or Force_regen == True :
    t1 = time.time()
    client = Config().get_client()
    FilesList = client.list(SOURCE_DIR)
    #FilesList = FilesList[:3] #Debug
    total = len(FilesList)

    ########## DOUBLE INDEX ####################
    rdd = sc.textFile(CLUSTER_ADD + MODEL_DIR + 'model.csv',gran*num_cores)
    rddDate = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[0]))
    rddAction = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[1]))
    
    I = [rddDate, rddAction]
    i=0
    header = ["Date", "Action"]
    for file_ in FilesList:
        i=i+1
        print("#########")
        print("Work: " + str(i) + "/" + str(total))

        rddFile = sc.textFile(CLUSTER_ADD + SOURCE_DIR + file_,gran*num_cores)
        header.append(rddFile.take(1)[0])

        rddT = rddFile.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,float(x)))

        I.append(rddT)

    
    t2 = time.time()
    print("####################################")
    print("TIME TAKEN FOR TASK:" + str(t2-t1))

    print("#############################")
    print("FILE LOADING TASK FINISHED")
    print("\n")
    
    #######################
    
    print("#############################")
    print("TASK: RDD to DF:")
    t1 = time.time()
    uni_ = sc.union(I)

    s = uni_.groupByKey().mapValues(tuple).map(lambda x: x[1])

    df = s.toDF(header)

    t2 = time.time()
    print("####################################")
    print("TIME TAKEN FOR TASK:" + str(t2-t1))

    print("#############################")
    print("RDD to DF TASK FINISHED")
    print("\n")
    
    ###################"
    
    print("#############################")
    print("TASK: Save Aggregated to HDFS:")
    t1 = time.time()
    df.write.parquet( CLUSTER_ADD + PARQUET_SOURCE_DIR + PARQUET_SOURCE_FILE)
    t2 = time.time()
    print("####################################")
    print("TIME TAKEN FOR TASK:" + str(t2-t1))

    print("#############################")
    print("Save Aggregated to HDFS TASK FINISHED")
    print("\n")
    
    check_integrity(df,FilesList)
    
else:
    print("LOADING PARQUET FILE DATAFRAME")
    t1 = time.time()
    df = spark.read.parquet( CLUSTER_ADD + PARQUET_SOURCE_DIR + PARQUET_SOURCE_FILE)
    t2 = time.time()
    print("####################################")
    print("TIME TAKEN FOR TASK:" + str(t2-t1))

    print("#############################")
    print("Parquet loading TASK FINISHED")
    print("\n")
    
print("Dataframe LOADED")

dfPersist = df.persist(StorageLevel.MEMORY_AND_DISK)


In [ ]:
dfPersist.show()

In [ ]:
dfPersist.count()

In [ ]:
t1 = time.time()
dfPersist.filter(dfPersist.Action == 'IQ103375').filter(dfPersist.Date < '2012-08-20')\
.filter(dfPersist.Date > '2010-08-20').orderBy("Date").select("Date", "Action").show()
t2 = time.time()

print t2 - t1

In [ ]:
rdd = sc.textFile(CLUSTER_ADD + MODEL_DIR + 'model.csv',4*num_cores)
print(rdd.count())
print(rdd.take(3))
rddDate = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[0]))
print rddDate.take(4)

rddAction = rdd.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,x)).map(lambda x: (x[0],x[1].split(",")[1]))
print rddAction.take(4)


In [ ]:
client = Config().get_client()
FilesList = client.list(SOURCE_DIR)

a = FilesList[0]
rdd2 = sc.textFile(CLUSTER_ADD + SOURCE_DIR + a,4*num_cores)
print(rdd2.count())
print(rdd2.take(3))
rdd2 = rdd2.zipWithIndex().filter(lambda ind: ind[1] > 0).map(lambda (x,y): (y,float(x)))
rdd2.take(4)



In [ ]:
u = [rddDate, rddAction, rdd2]
u

In [ ]:
te = sc.union(u)

#tup0 = index created
#tup1 = data
#map data to float 
st = te.groupByKey().mapValues(tuple).map(lambda x: x[1])
st.take(4)


In [ ]:
rddDate.count()

In [ ]:
rdd2.count()

In [ ]:
dft = st.toDF(["date", "action" ,"X1"])
dft.orderBy("date").show()

In [ ]:
h1 = Mat3.columns[125]
h2 = Mat3.columns[302]

In [ ]:
Mat3.stat.corr(h1,h2)

In [ ]:
dfPersist.rdd

In [ ]:
Mat3.write.parquet(CLUSTER_ADD+"/user/cluster/dataFSave.parquet")

In [ ]:
a = s.map(lambda x: (1, len(x))).countByValue()



In [ ]:
header = []
I = []

for file_ in FileList:
    # CHARGEMENT DU FICHIER CSV
    rddFile = sc.textFile(CLUSTER_ADD + SOURCE_DIR + file_,gran*num_cores)
    
    # On récupère le nom de la variable (1e ligne)
    header.append(rddFile.take(1)[0])

    rddT = rddFile.zipWithIndex()\ ### AJOUTE UN INDEX FICTIF de 1 à 3M
                    .filter(lambda ind: ind[1] > 0)\ ### On retire la 1e ligne (nom de la variable)
                    .map(lambda (x,y): (y,float(x))) ### On inverse les 2 colonnes et on force la conversion en float

    I.append(rddT) # On met dans la liste
    
uni_ = sc.union(I) ### Concatène les RDD, list[RDD] -> UnionRDD[float]

s = uni_.groupByKey()\ ### On regroupe par clé, ici l'index on obtient RDD (key => Iterator[X1,X2...,X465,X466])
    .mapValues(tuple)\ ### On transforme l'itérateur en tuple, RDD = (key,(X1,...,X466))
    .map(lambda x: x[1]) ### On conserve uniquement le tuple
    
df = s.toDF(header) ### Conversion en Dataframe

#### 2 Transformations: GroupByKey (shuffle transfo) et toDF (normal transfo)

In [ ]:
df = dfPersist.select(h[2:])
df.select(h[2:]).rdd.map(lambda data: Vectors.dense([float(c) for c in data]))

df.take(2)


In [ ]:
t1 = time.time()
h = dfPersist.columns
rddTest = dfPersist.select(h[2:]).rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
A = Statistics.corr(rddTest)
A
        
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

In [ ]:
A

In [ ]:
file_ = FilesList[0]


file_

In [ ]:
import numpy as np
import pandas as pd

df1 = pd.read_csv(CLUSTER_ADD + SOURCE_DIR + file_)
df1.head()




In [ ]:
import random
client = Config().get_client()
FilesList = client.list(SOURCE_DIR)
header = dfPersist.columns
seed = time.time()
random.seed(seed)
num_check = int(len(FilesList) * 1 / 10)
print(num_check)

for i in range(len(FilesList)):
    id = random.randint(0,len(FilesList) - 1)
    print("WORK: " + str(i) + " ; " + str(id))
    
    with client.read(SOURCE_DIR + FilesList[i]) as reader:
        dfPandas = pd.read_csv(reader)
    
    dfCheck = dfPersist.orderBy("Date", "Action").select([header[2+i]]).toPandas()
    assert dfPandas.equals(dfCheck), "Integrity Check Failure: " + str(header[2+id]) + "seed: " + str(seed)

In [ ]:
dfCheck = dfPersist.orderBy("Date", "Action").select([header[2+id]]).toPandas()

In [ ]:
#dfPandas.sort_values(header[2+id],ascending=False)
dfPandas.head()

In [ ]:
dfCheck.head()

In [ ]:
assert dfPandas.equals(dfCheck), "Integrity Check Failure: " + str(header[2+id]) + "seed: " + str(seed)

In [ ]:
client = Config().get_client()
FilesList = client.list(SOURCE_DIR)
header = dfPersist.columns
t1 = time.time()
columns = header[2:]
t = len(columns)

MatCheck = A

for i in range(t):
    print("Line " + str(i))
    with client.read(SOURCE_DIR + FilesList[i]) as reader:
        dfP1 = pd.read_csv(reader)
    for j in range(i+1,t):
        if (j % 100 == 0):
            print("     Col " + str(j))
        with client.read(SOURCE_DIR + FilesList[j]) as reader:
            dfP2 = pd.read_csv(reader)
            corr = dfP2.corrwith(dfP1[header[2+i]])
            assert ( (MatCheck[i,j] - corr[0]) < 10e-5), "Correlation Failure: (i,j): (" + str(i) + ";" + str(j) +"); delta = " + str(MatCheck[i,j] - corr[0])         
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))


In [ ]:
import gc

gc.collect()

In [ ]:
del dfPandas
del dfCheck

In [ ]:
dfPersist.select(header[1]).withColumn("test",str("Action") + str(col)).show()

In [ ]:
col = dfPersist.Action
col